In [ ]:
import pandas_profiling
import datetime
from datetime import timedelta
import time
import pandas as pd
import matplotlib as plt
import seaborn as sns
import json
import numpy as np
from scipy import stats

In [ ]:
## Import files:

earthquake_df = pd.read_csv('usgs_mag5_1990_2019.csv')
aqs_df = pd.read_csv('final_aq.csv')

In [ ]:
#We have a large enoug N to drop the NA measurements 
aqs_df.dropna()

In [ ]:
sns.pairplot(data=aqs_df, hue = 'Parameter', plot_kws={'s':3})

In [ ]:
# I need to figure out a way to calculate the day distance to the even from each measurement:
# event_date - measurement date = time_delta could be a time delta or an integer but just day. 15 bins [-7,-6,...,-1,0,1,...6,7]
#earthquake_df = earthquake_df.drop(index='Unnamed: 0')
aqs_df.keys()

In [ ]:
event_date = earthquake_df[['Identifier', 'Date']]
aqs_df = aqs_df.rename(columns={'q_ueid':'Identifier'})
master_record = aqs_df.merge(earthquake_df, on='Identifier', suffixes=('_measure', '_event'))

# raw_data['Mycol'] =  pd.to_datetime(raw_data['Mycol'], format='%d%b%Y:%H:%M:%S.%f
master_record['Date_measure'] = pd.to_datetime(master_record['Date_measure'])
master_record['Date_event'] = pd.to_datetime(master_record['Date_event'], format='%Y%m%d')
master_record['time_bins'] =(master_record['Date_measure']-master_record['Date_event']).dt.days
clean_record = master_record[['Parameter','Measurement', 'Unit', 'Station', 'County', 'Location', 'GCS', 'Depth', 'Magnitude', 'Date_measure', 'Date_event', 'time_bins']]
pollutant_list = clean_record['Parameter'].unique().tolist()
pollutant_list

In [ ]:
clean_record.to_csv('final_epa_usgs_df.csv')

In [ ]:
#Iterate over this to make graphs for each pollutant:
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.stripplot(x=clean_record['time_bins'], y = clean_record['Measurement'], hue=clean_record['Station'], data = clean_record[clean_record['Parameter'] == 'PM2.5 - Local Conditions'], marker='.', s=1, )
ax.set(ylim=(0, 200))
ax.legend().set_visible(False)



In [ ]:
#g = sns.FacetGrid(clean_record, col='Parameter', hue='Unit', col_wrap=5)
sns.set(rc={'figure.figsize':(11.7,8.27)})
#Iterate over this to make graphs for each pollutant:
ax = sns.violinplot(x=clean_record['time_bins'], y = clean_record['Measurement'], data = clean_record[clean_record['Parameter'] == 'PM2.5 - Local Conditions'], marker='.', s=1, aspect=1.75)
ax.set(ylim=(-5, 20))
ax

In [ ]:
clean_record.to_csv('final_epa_usgs_df.csv')

In [ ]:
clean_record = clean_record[clean_record['Parameter']=='PM2.5 - Local Conditions']

In [ ]:
#List of data by days:
clean_record = clean_record.dropna()
bins = clean_record['time_bins'].unique().tolist()
bin_breakdown = [clean_record[clean_record['time_bins']==x] for x in bins]
len(bin_breakdown)
index = 0
ks_stat_dic = {}
# This calculates a pairwise kolmogorov smirnov test matrix for use in a heat map later
for dep_var in bins:
    for ind_var in bins:
        cutoff= min(len(bin_breakdown[dep_var]['Measurement']),len(bin_breakdown[ind_var]['Measurement']))
        ks_stat, tt_pv = stats.ks_2samp(bin_breakdown[dep_var]["Measurement"].sample(n=cutoff), bin_breakdown[ind_var]["Measurement"].sample(n=cutoff))
        ks_stat_dic[str(index)] =  {'Bin_1': dep_var,
                               'Bin_2': ind_var, 
                               'KS Stat': ks_stat,
                               'TT p-val':tt_pv}
        #status = f'{ind_var} vs: {dep_var}: KS Stat: {np.round(ks_stat, decimals=6)}, Two Taile p-value: {np.round(tt_pv, decimals=5)}'
        index +=1
        #print(index)
#ks_stat_dic

In [ ]:
## Calculating the Kolmogorov Smirnov Matrix
ks_matrix = pd.DataFrame.from_dict(ks_stat_dic)
ks_matrix = ks_matrix.T
ks_matrix = ks_matrix.pivot('Bin_1', 'Bin_2', 'TT p-val')
ks_matrix



In [ ]:
r = sns.heatmap(data=ks_matrix, cmap='coolwarm', square=True, robust=True, annot=True, fmt='.3f', vmin=0, vmax=0.05)
r.set_ylim(0,15)
r.set_xlim(0,15)